Copyright 2018 Google LLC

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

# Experiment 1: fixed detector in many scenarios

This notebook computes the performance of the fixed strategies in various scenarios. This experiment is described in Sec. 5.2 of CVPR submission "Learning Intelligent Dialogs for Bounding Box Annotation".

In [0]:
import matplotlib.pyplot as plt
import numpy as np
from __future__ import division
from __future__ import print_function
import math
import gym
from gym import spaces

import pandas as pd

from sklearn import linear_model, ensemble, neural_network, model_selection, ensemble
from sklearn.neural_network import MLPClassifier

from third_party import np_box_ops
import annotator, detector, dialog, environment

To specify the experiments, 3 paramters need to be defined: 

*   detector
*   type of drawing
*   desired quality of bounding boxes (this notebook only works with strong detections)

All together, it gives 8 possible experiment, 6 of which were presented in the paper.

In [0]:
# desired quality: high (min_iou=0.7) and low (min_iou=0.5)
min_iou = 0.7 # @param ["0.5", "0.7"]
# drawing speed: high (time_draw=7) and low (time_draw=25)
time_draw = 7 # @param ["7", "25"]

# if detector is weak, then we use best MIL, if it is strong, we use detector trained on PASCAL 2012
detector_weak = False # @param ["False"]

Other parameters of the experiment

In [0]:
random_seed = 805 # global variable that fixes the random seed everywhere for replroducibility of results

# what kind of features will be used to represent the state
# numerical values 1-20 correspond to one hot encoding of class
predictive_fields = ['prediction_score', 'relative_size', 'avg_score', 'dif_avg_score', 'dif_max_score', 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]

time_verify = 1.8 # @param

# select one of the 10 folds
fold = 8 # @param

# Load all data

In [0]:
# Download GT:
# wget wget https://storage.googleapis.com/iad_pascal_annotations_and_detections/pascal_gt_for_iad.h5
# Download detections with features
# wget https://storage.googleapis.com/iad_pascal_annotations_and_detections/pascal_proposals_plus_features_for_iad.h5

download_dir = ''
ground_truth = pd.read_hdf(download_dir + 'pascal_gt_for_iad.h5', 'ground_truth')
box_proposal_features = pd.read_hdf(download_dir + 'pascal_proposals_plus_features_for_iad.h5', 'box_proposal_features')

# Initialise the experiment

In [0]:
the_annotator = annotator.AnnotatorSimple(ground_truth, random_seed, time_verify, time_draw, min_iou)
the_detector = detector.Detector(box_proposal_features, predictive_fields)

In [0]:
image_class = ground_truth[['image_id', 'class_id']]
image_class = image_class.drop_duplicates()

Select the trainig and testing data according to the selected fold. We split all images in 10 approximately equal parts and each fold includes these images together with all classes present in them.

In [0]:
# get a list of unique images
unique_image = image_class['image_id'].drop_duplicates()
# a list of image+class pairs
image_class_array = image_class.values[:,0]


if fold==1:
  index_image_class1 = 0
else:
  image_division1 = unique_image.iloc[502+501*(fold-2)]
  index_image_class1 = np.searchsorted(image_class_array, image_division1, side='right')
  
if fold==10:
  index_image_class2 = len(image_class_array)
else:
  image_division2 = unique_image.iloc[502+501*(fold-1)]
  index_image_class2 = np.searchsorted(image_class_array, image_division2, side='right')

# the selected fold becomes the training set
image_class_train = image_class.iloc[index_image_class1:index_image_class2]
# the other 9 folds become test set
image_class_test = pd.concat([image_class.iloc[0:index_image_class1],image_class.iloc[index_image_class2:]])

Initialise the environment for testing the strategies.

In [0]:
env_train = environment.AnnotatingDataset(the_annotator, the_detector, image_class_train)
env_test = environment.AnnotatingDataset(the_annotator, the_detector, image_class_test)

# Experiment with fixed strategies

In [0]:
lower_bound = False
# possible values: DialogD, DialogV, DialogV1D, DialogV2D, DialogV3D, best
if lower_bound:
  num_verifications = int(math.floor(env_train.annotator.time_draw/env_train.annotator.time_verify))
else:
  num_verifications = 2 # @param

In [10]:
%output_height 300

print('Running ', len(env_test.image_class), 'episodes with strategy: V', str(num_verifications), 'D')

# total reward of all annotation episodes
total_reward = 0
all_rewards = []
# go through all training image+class pairs
for i in range(len(env_test.image_class)):
  print('Episode ', i, end = ': ')
  # create an agent that generates dialogs
  agent = dialog.FixedDialog(num_verifications)
  # resent the enviroment to set the current imag+class pair to be at index i
  state = env_test.reset(current_index=i)
  done = False
  episode_reward = 0
  # until the end of the episode is reached
  while not(done):
    action = agent.get_next_action(state)
    if action==0:
      print('V', end='')
    elif action==1:
      print('D', end='')
    # make the next step
    next_state, reward, done, _ = env_test.step(action)
    state = next_state
    total_reward += reward
    episode_reward += reward
  all_rewards.append(episode_reward)
  print()
  
if lower_bound:
  all_rewards = np.array(all_rewards)
  too_long = (all_rewards < -env_test.annotator.time_draw)
  all_rewards[too_long] = -env_test.annotator.time_draw
  total_reward = sum(all_rewards)
    
print('Total duration of all episodes = ', -total_reward)    
print('Average episode duration = ', -total_reward/len(env_test.image_class))

Running  7137 episodes with strategy: V 2 D
Episode  0: VV
Episode  1: V
Episode  2: V
Episode  3: V
Episode  4: V
Episode  5: V
Episode  6: V
Episode  7: V
Episode  8: V
Episode  9: V
Episode  10: VVD
Episode  11: V
Episode  12: V
Episode  13: V
Episode  14: V
Episode  15: V
Episode  16: V
Episode  17: V
Episode  18: VVD
Episode  19: V
Episode  20: V
Episode  21: V
Episode  22: VVD
Episode  23: V
Episode  24: V
Episode  25: V
Episode  26: V
Episode  27: V
Episode  28: V
Episode  29: V
Episode  30: V
Episode  31: V
Episode  32: V
Episode  33: VVD
Episode  34: V
Episode  35: VV
Episode  36: V
Episode  37: VVD
Episode  38: V
Episode  39: V
Episode  40: VVD
Episode  41: VVD
Episode  42: VV
Episode  43: V
Episode  44: V
Episode  45: V
Episode  46: V
Episode  47: V
Episode  48: V
Episode  49: V
Episode  50: VVD
Episode  51: V
Episode  52: V
Episode  53: VVD
Episode  54: V
Episode  55: V
Episode  56: VVD
Episode  57: V
Episode  58: V
Episode  59: VVD
Episode  60: V
Episode  61: V
Episode  62

Episode  506: V
Episode  507: V
Episode  508: V
Episode  509: V
Episode  510: VVD
Episode  511: VVD
Episode  512: V
Episode  513: V
Episode  514: V
Episode  515: V
Episode  516: VV
Episode  517: V
Episode  518: V
Episode  519: VVD
Episode  520: V
Episode  521: V
Episode  522: V
Episode  523: VVD
Episode  524: V
Episode  525: V
Episode  526: V
Episode  527: V
Episode  528: VVD
Episode  529: V
Episode  530: V
Episode  531: V
Episode  532: V
Episode  533: V
Episode  534: V
Episode  535: V
Episode  536: VVD
Episode  537: V
Episode  538: V
Episode  539: V
Episode  540: V
Episode  541: V
Episode  542: V
Episode  543: V
Episode  544: V
Episode  545: VVD
Episode  546: V
Episode  547: V
Episode  548: V
Episode  549: V
Episode  550: V
Episode  551: V
Episode  552: V
Episode  553: V
Episode  554: VVD
Episode  555: V
Episode  556: V
Episode  557: V
Episode  558: V
Episode  559: V
Episode  560: V
Episode  561: V
Episode  562: V
Episode  563: V
Episode  564: V
Episode  565: V
Episode  566: V
Episode

Episode  1010: V
Episode  1011: V
Episode  1012: V
Episode  1013: VVD
Episode  1014: V
Episode  1015: V
Episode  1016: V
Episode  1017: V
Episode  1018: V
Episode  1019: V
Episode  1020: V
Episode  1021: VVD
Episode  1022: V
Episode  1023: V
Episode  1024: V
Episode  1025: V
Episode  1026: V
Episode  1027: V
Episode  1028: V
Episode  1029: V
Episode  1030: VVD
Episode  1031: VV
Episode  1032: V
Episode  1033: V
Episode  1034: VVD
Episode  1035: VVD
Episode  1036: V
Episode  1037: VV
Episode  1038: V
Episode  1039: V
Episode  1040: V
Episode  1041: V
Episode  1042: V
Episode  1043: V
Episode  1044: V
Episode  1045: V
Episode  1046: V
Episode  1047: V
Episode  1048: VVD
Episode  1049: V
Episode  1050: V
Episode  1051: V
Episode  1052: VVD
Episode  1053: VVD
Episode  1054: V
Episode  1055: VVD
Episode  1056: V
Episode  1057: V
Episode  1058: V
Episode  1059: VVD
Episode  1060: VVD
Episode  1061: V
Episode  1062: VVD
Episode  1063: VV
Episode  1064: V
Episode  1065: V
Episode  1066: V
Epis

Episode  1511: V
Episode  1512: V
Episode  1513: V
Episode  1514: V
Episode  1515: V
Episode  1516: VVD
Episode  1517: V
Episode  1518: V
Episode  1519: V
Episode  1520: VVD
Episode  1521: VVD
Episode  1522: V
Episode  1523: V
Episode  1524: V
Episode  1525: V
Episode  1526: V
Episode  1527: V
Episode  1528: V
Episode  1529: V
Episode  1530: V
Episode  1531: VVD
Episode  1532: V
Episode  1533: V
Episode  1534: V
Episode  1535: V
Episode  1536: V
Episode  1537: V
Episode  1538: V
Episode  1539: V
Episode  1540: V
Episode  1541: VV
Episode  1542: V
Episode  1543: V
Episode  1544: V
Episode  1545: V
Episode  1546: V
Episode  1547: V
Episode  1548: VVD
Episode  1549: V
Episode  1550: V
Episode  1551: V
Episode  1552: V
Episode  1553: V
Episode  1554: VVD
Episode  1555: V
Episode  1556: V
Episode  1557: V
Episode  1558: V
Episode  1559: VV
Episode  1560: V
Episode  1561: V
Episode  1562: V
Episode  1563: V
Episode  1564: VVD
Episode  1565: VVD
Episode  1566: VVD
Episode  1567: V
Episode  15

Episode  2003: V
Episode  2004: V
Episode  2005: V
Episode  2006: V
Episode  2007: VVD
Episode  2008: V
Episode  2009: V
Episode  2010: V
Episode  2011: V
Episode  2012: VVD
Episode  2013: V
Episode  2014: VV
Episode  2015: V
Episode  2016: V
Episode  2017: VV
Episode  2018: V
Episode  2019: VV
Episode  2020: V
Episode  2021: VVD
Episode  2022: V
Episode  2023: V
Episode  2024: V
Episode  2025: V
Episode  2026: V
Episode  2027: V
Episode  2028: V
Episode  2029: V
Episode  2030: V
Episode  2031: V
Episode  2032: V
Episode  2033: V
Episode  2034: VVD
Episode  2035: V
Episode  2036: VVD
Episode  2037: V
Episode  2038: V
Episode  2039: V
Episode  2040: VVD
Episode  2041: V
Episode  2042: V
Episode  2043: V
Episode  2044: V
Episode  2045: V
Episode  2046: V
Episode  2047: VVD
Episode  2048: V
Episode  2049: V
Episode  2050: V
Episode  2051: V
Episode  2052: V
Episode  2053: V
Episode  2054: V
Episode  2055: V
Episode  2056: VVD
Episode  2057: V
Episode  2058: VVD
Episode  2059: V
Episode  2

...

Episode  4952: V
Episode  4953: VVD
Episode  4954: VVD
Episode  4955: V
Episode  4956: V
Episode  4957: V
Episode  4958: VVD
Episode  4959: V
Episode  4960: V
Episode  4961: VVD
Episode  4962: VVD
Episode  4963: VV
Episode  4964: V
Episode  4965: V
Episode  4966: V
Episode  4967: VVD
Episode  4968: V
Episode  4969: V
Episode  4970: V
Episode  4971: VVD
Episode  4972: V
Episode  4973: V
Episode  4974: VVD
Episode  4975: V
Episode  4976: V
Episode  4977: V
Episode  4978: V
Episode  4979: V
Episode  4980: V
Episode  4981: V
Episode  4982: V
Episode  4983: VV
Episode  4984: V
Episode  4985: V
Episode  4986: V
Episode  4987: V
Episode  4988: V
Episode  4989: V
Episode  4990: V
Episode  4991: V
Episode  4992: V
Episode  4993: VVD
Episode  4994: V
Episode  4995: V
Episode  4996: V
Episode  4997: V
Episode  4998: V
Episode  4999: VVD
Episode  5000: VV
Episode  5001: V
Episode  5002: VVD
Episode  5003: V
Episode  5004: V
Episode  5005: V
Episode  5006: V
Episode  5007: VV
Episode  5008: V
Episo

Episode  5461: VVD
Episode  5462: V
Episode  5463: V
Episode  5464: V
Episode  5465: V
Episode  5466: V
Episode  5467: V
Episode  5468: V
Episode  5469: V
Episode  5470: V
Episode  5471: V
Episode  5472: V
Episode  5473: V
Episode  5474: VV
Episode  5475: V
Episode  5476: V
Episode  5477: V
Episode  5478: V
Episode  5479: V
Episode  5480: V
Episode  5481: V
Episode  5482: V
Episode  5483: V
Episode  5484: V
Episode  5485: V
Episode  5486: V
Episode  5487: V
Episode  5488: V
Episode  5489: V
Episode  5490: V
Episode  5491: V
Episode  5492: VVD
Episode  5493: V
Episode  5494: V
Episode  5495: V
Episode  5496: V
Episode  5497: V
Episode  5498: V
Episode  5499: VVD
Episode  5500: V
Episode  5501: VVD
Episode  5502: V
Episode  5503: V
Episode  5504: VVD
Episode  5505: V
Episode  5506: V
Episode  5507: V
Episode  5508: VV
Episode  5509: VVD
Episode  5510: VV
Episode  5511: VV
Episode  5512: V
Episode  5513: VVD
Episode  5514: VVD
Episode  5515: V
Episode  5516: V
Episode  5517: V
Episode  55

Episode  5959: VVD
Episode  5960: V
Episode  5961: VVD
Episode  5962: V
Episode  5963: V
Episode  5964: V
Episode  5965: VVD
Episode  5966: V
Episode  5967: VVD
Episode  5968: V
Episode  5969: V
Episode  5970: V
Episode  5971: V
Episode  5972: V
Episode  5973: V
Episode  5974: V
Episode  5975: V
Episode  5976: V
Episode  5977: V
Episode  5978: V
Episode  5979: V
Episode  5980: V
Episode  5981: V
Episode  5982: V
Episode  5983: V
Episode  5984: V
Episode  5985: V
Episode  5986: V
Episode  5987: V
Episode  5988: V
Episode  5989: VVD
Episode  5990: VVD
Episode  5991: V
Episode  5992: V
Episode  5993: V
Episode  5994: V
Episode  5995: V
Episode  5996: V
Episode  5997: VVD
Episode  5998: V
Episode  5999: V
Episode  6000: V
Episode  6001: V
Episode  6002: V
Episode  6003: V
Episode  6004: VVD
Episode  6005: V
Episode  6006: V
Episode  6007: VVD
Episode  6008: V
Episode  6009: VVD
Episode  6010: V
Episode  6011: V
Episode  6012: V
Episode  6013: V
Episode  6014: V
Episode  6015: V
Episode  60

Episode  6460: V
Episode  6461: V
Episode  6462: VVD
Episode  6463: V
Episode  6464: V
Episode  6465: V
Episode  6466: V
Episode  6467: V
Episode  6468: V
Episode  6469: V
Episode  6470: V
Episode  6471: V
Episode  6472: V
Episode  6473: VVD
Episode  6474: V
Episode  6475: V
Episode  6476: VVD
Episode  6477: VV
Episode  6478: V
Episode  6479: V
Episode  6480: V
Episode  6481: V
Episode  6482: V
Episode  6483: V
Episode  6484: V
Episode  6485: V
Episode  6486: V
Episode  6487: V
Episode  6488: V
Episode  6489: V
Episode  6490: V
Episode  6491: V
Episode  6492: V
Episode  6493: V
Episode  6494: VVD
Episode  6495: V
Episode  6496: V
Episode  6497: V
Episode  6498: V
Episode  6499: V
Episode  6500: V
Episode  6501: V
Episode  6502: V
Episode  6503: V
Episode  6504: V
Episode  6505: V
Episode  6506: V
Episode  6507: V
Episode  6508: V
Episode  6509: V
Episode  6510: VVD
Episode  6511: V
Episode  6512: V
Episode  6513: V
Episode  6514: V
Episode  6515: V
Episode  6516: V
Episode  6517: V
Epi

Episode  6950: V
Episode  6951: VVD
Episode  6952: VVD
Episode  6953: V
Episode  6954: V
Episode  6955: VVD
Episode  6956: V
Episode  6957: V
Episode  6958: V
Episode  6959: V
Episode  6960: V
Episode  6961: V
Episode  6962: V
Episode  6963: V
Episode  6964: VVD
Episode  6965: V
Episode  6966: V
Episode  6967: V
Episode  6968: V
Episode  6969: V
Episode  6970: V
Episode  6971: V
Episode  6972: VVD
Episode  6973: V
Episode  6974: VV
Episode  6975: V
Episode  6976: V
Episode  6977: V
Episode  6978: V
Episode  6979: V
Episode  6980: V
Episode  6981: V
Episode  6982: V
Episode  6983: VVD
Episode  6984: V
Episode  6985: VVD
Episode  6986: VVD
Episode  6987: V
Episode  6988: V
Episode  6989: V
Episode  6990: V
Episode  6991: V
Episode  6992: V
Episode  6993: V
Episode  6994: V
Episode  6995: V
Episode  6996: V
Episode  6997: V
Episode  6998: V
Episode  6999: V
Episode  7000: V
Episode  7001: V
Episode  7002: V
Episode  7003: VVD
Episode  7004: V
Episode  7005: V
Episode  7006: V
Episode  700

# Experiments with IAD-Prob 

## Collect examples of episodes on the training part of data

In [11]:
%output_height 300

print('Running ', len(env_train.image_class), 'episodes with strategy V3D for data collection')

# total reward of all annotation episodes
total_reward = 0
data_for_classifier = []
# go through all training image+class pairs
for i in range(len(env_train.image_class)):
  print('Episode ', i, end = ': ')
  # create an agent that generates dialogs
  # for collecting data we usually use strategy V3D
  # however, I think that if the cases when longer verification series are better, it is better to collect data longer verification series
  agent = dialog.FixedDialog(5)
  # resent the enviroment to set the current imag+class pair to be at index i
  state = env_train.reset(current_index=i)
  done = False
  # until the end of the episode is reached
  while not(done):
    action = agent.get_next_action(state)
    # make the next step
    next_state, reward, done, _ = env_train.step(action)
    if action==0:
      print('V', end='')
      state_dict = dict(state)
      state_dict['is_accepted'] = done
      data_for_classifier.append(state_dict)
    elif action==1:
      print('D', end='')
    state = next_state
    total_reward += reward
  print()
  
print('Total duration of all episodes = ', -total_reward)    
print('Average episode duration = ', -total_reward/len(env_train.image_class))

data_for_classifier = pd.DataFrame(data_for_classifier)

Running  776 episodes with strategy V3D for data collection
Episode  0: VVVVVD
Episode  1: V
Episode  2: VV
Episode  3: V
Episode  4: VVVVVD
Episode  5: V
Episode  6: V
Episode  7: V
Episode  8: V
Episode  9: VVVVVD
Episode  10: VVVVVD
Episode  11: V
Episode  12: VVVVVD
Episode  13: V
Episode  14: VVVV
Episode  15: V
Episode  16: V
Episode  17: V
Episode  18: VVVVVD
Episode  19: V
Episode  20: V
Episode  21: VVVVVD
Episode  22: V
Episode  23: VV
Episode  24: V
Episode  25: VVVVVD
Episode  26: VVVVVD
Episode  27: V
Episode  28: V
Episode  29: V
Episode  30: VVVVVD
Episode  31: V
Episode  32: VV
Episode  33: V
Episode  34: V
Episode  35: V
Episode  36: V
Episode  37: V
Episode  38: VVVVVD
Episode  39: V
Episode  40: V
Episode  41: V
Episode  42: V
Episode  43: V
Episode  44: V
Episode  45: VVVVVD
Episode  46: VVVVVD
Episode  47: V
Episode  48: V
Episode  49: VVVVVD
Episode  50: V
Episode  51: V
Episode  52: V
Episode  53: V
Episode  54: VV
Episode  55: VVVVV
Episode  56: V
Episode  57: V

Episode  653: V
Episode  654: V
Episode  655: V
Episode  656: V
Episode  657: V
Episode  658: V
Episode  659: VVV
Episode  660: VVVVV
Episode  661: V
Episode  662: VVVVVD
Episode  663: V
Episode  664: V
Episode  665: V
Episode  666: V
Episode  667: V
Episode  668: V
Episode  669: V
Episode  670: V
Episode  671: V
Episode  672: V
Episode  673: VVV
Episode  674: V
Episode  675: VVVVVD
Episode  676: V
Episode  677: V
Episode  678: V
Episode  679: V
Episode  680: V
Episode  681: V
Episode  682: V
Episode  683: VVVVVD
Episode  684: V
Episode  685: V
Episode  686: V
Episode  687: V
Episode  688: V
Episode  689: V
Episode  690: V
Episode  691: V
Episode  692: VVVVVD
Episode  693: VVVVVD
Episode  694: V
Episode  695: VVV
Episode  696: V
Episode  697: V
Episode  698: V
Episode  699: V
Episode  700: VVVVVD
Episode  701: V
Episode  702: V
Episode  703: V
Episode  704: V
Episode  705: V
Episode  706: V
Episode  707: V
Episode  708: V
Episode  709: V
Episode  710: V
Episode  711: V
Episode  712: V


## Find the best classification model to predict if a box is going to be accepted or rejected

In [12]:
np.random.seed(random_seed) # for reproducibility of fitting the classifier and cross-validation

print('Cross-validating parameters\' values... This might take some time.')

# possible parameter values
parameters = {'hidden_layer_sizes': ((20, 20, 20), (50, 50, 50), (80, 80, 80), (20, 20, 20, 20), (50, 50, 50, 50), (80, 80, 80, 80), (20, 20, 20, 20, 20), (50, 50, 50, 50, 50), (80, 80, 80, 80, 80)), 'activation': ('logistic', 'relu'), 'alpha': [0.0001, 0.001, 0.01]}
model_mlp = neural_network.MLPClassifier()
# cross-validate parameters
grid_search = model_selection.GridSearchCV(model_mlp, parameters, scoring='neg_log_loss', refit=True)
grid_search.fit(data_for_classifier[predictive_fields], data_for_classifier['is_accepted'])
print('best score = ', grid_search.best_score_)
print('best parameters = ', grid_search.best_params_)
# use the model with the best parameters
acceptance_prediction_model = grid_search.best_estimator_

Cross-validating parameters' values... This might take some time.
best score =  -0.429081211862
best parameters =  {'hidden_layer_sizes': (50, 50, 50), 'activation': 'logistic', 'alpha': 0.01}


## Run the experiment with IAD-Prob

In [13]:
%output_height 300

# initialise the agent
agent = dialog.DialogProb(acceptance_prediction_model, the_annotator)

print('Running ', len(env_test.image_class), 'episodes with strategy IAD-Prob')
# total reward of all annotation episodes
total_reward = 0
# go through all test image+class pairs
for i in range(len(env_test.image_class)):
  print('Episode ', i, end = ': ')
  # resent the enviroment and select the nest image+class pair
  state = env_test.reset(current_index=i)
  done = False
  # until the end of the episode is reached
  while not(done):
    action = agent.get_next_action(state)
    if action==0:
      print('V', end='')
    elif action==1:
      print('D', end='')
    # make the next step
    next_state, reward, done, _ = env_test.step(action)
    state = next_state
    total_reward += reward
  print()

    
print('Total duration of all episodes = ', -total_reward)    
print('Average episode duration = ', -total_reward/len(env_test.image_class))

Running  7137 episodes with strategy IAD-Prob
Episode  0: VV
Episode  1: V
Episode  2: V
Episode  3: V
Episode  4: V
Episode  5: V
Episode  6: V
Episode  7: V
Episode  8: V
Episode  9: V
Episode  10: VD
Episode  11: V
Episode  12: V
Episode  13: V
Episode  14: V
Episode  15: V
Episode  16: V
Episode  17: V
Episode  18: VVV
Episode  19: V
Episode  20: V
Episode  21: V
Episode  22: D
Episode  23: V
Episode  24: V
Episode  25: V
Episode  26: V
Episode  27: V
Episode  28: V
Episode  29: V
Episode  30: V
Episode  31: V
Episode  32: V
Episode  33: D
Episode  34: V
Episode  35: VD
Episode  36: V
Episode  37: D
Episode  38: V
Episode  39: V
Episode  40: D
Episode  41: VVVVD
Episode  42: VV
Episode  43: V
Episode  44: V
Episode  45: V
Episode  46: V
Episode  47: V
Episode  48: V
Episode  49: V
Episode  50: D
Episode  51: V
Episode  52: D
Episode  53: VVD
Episode  54: V
Episode  55: V
Episode  56: VVV
Episode  57: V
Episode  58: V
Episode  59: VVD
Episode  60: V
Episode  61: V
Episode  62: D
Epi

Episode  733: VD
Episode  734: V
Episode  735: V
Episode  736: V
Episode  737: VD
Episode  738: VD
Episode  739: V
Episode  740: V
Episode  741: V
Episode  742: VD
Episode  743: V
Episode  744: V
Episode  745: V
Episode  746: V
Episode  747: VVV
Episode  748: V
Episode  749: D
Episode  750: D
Episode  751: VVVVV
Episode  752: VD
Episode  753: V
Episode  754: V
Episode  755: V
Episode  756: V
Episode  757: VV
Episode  758: V
Episode  759: V
Episode  760: V
Episode  761: V
Episode  762: VVD
Episode  763: V
Episode  764: V
Episode  765: V
Episode  766: V
Episode  767: V
Episode  768: V
Episode  769: V
Episode  770: V
Episode  771: VD
Episode  772: V
Episode  773: V
Episode  774: V
Episode  775: V
Episode  776: V
Episode  777: V
Episode  778: V
Episode  779: V
Episode  780: V
Episode  781: D
Episode  782: VD
Episode  783: D
Episode  784: V
Episode  785: V
Episode  786: V
Episode  787: VD
Episode  788: V
Episode  789: V
Episode  790: VV
Episode  791: V
Episode  792: V
Episode  793: V
Episod

Episode  1436: V
Episode  1437: V
Episode  1438: V
Episode  1439: V
Episode  1440: V
Episode  1441: V
Episode  1442: V
Episode  1443: V
Episode  1444: VVVVD
Episode  1445: V
Episode  1446: V
Episode  1447: V
Episode  1448: V
Episode  1449: D
Episode  1450: D
Episode  1451: V
Episode  1452: V
Episode  1453: V
Episode  1454: V
Episode  1455: VVD
Episode  1456: V
Episode  1457: VVD
Episode  1458: V
Episode  1459: V
Episode  1460: VD
Episode  1461: V
Episode  1462: V
Episode  1463: V
Episode  1464: V
Episode  1465: V
Episode  1466: V
Episode  1467: V
Episode  1468: VV
Episode  1469: V
Episode  1470: V
Episode  1471: V
Episode  1472: V
Episode  1473: V
Episode  1474: V
Episode  1475: VVD
Episode  1476: V
Episode  1477: V
Episode  1478: V
Episode  1479: V
Episode  1480: V
Episode  1481: VD
Episode  1482: V
Episode  1483: V
Episode  1484: V
Episode  1485: V
Episode  1486: V
Episode  1487: D
Episode  1488: V
Episode  1489: V
Episode  1490: V
Episode  1491: V
Episode  1492: V
Episode  1493: V
E

Episode  1939: V
Episode  1940: V
Episode  1941: V
Episode  1942: VD
Episode  1943: V
Episode  1944: V
Episode  1945: V
Episode  1946: V
Episode  1947: V
Episode  1948: VD
Episode  1949: D
Episode  1950: VD
Episode  1951: V
Episode  1952: D
Episode  1953: VD
Episode  1954: V
Episode  1955: V
Episode  1956: V
Episode  1957: V
Episode  1958: V
Episode  1959: V
Episode  1960: VD
Episode  1961: V
Episode  1962: VVD
Episode  1963: V
Episode  1964: V
Episode  1965: V
Episode  1966: V
Episode  1967: V
Episode  1968: V
Episode  1969: VD
Episode  1970: V
Episode  1971: VD
Episode  1972: V
Episode  1973: VVD
Episode  1974: V
Episode  1975: V
Episode  1976: V
Episode  1977: V
Episode  1978: V
Episode  1979: V
Episode  1980: V
Episode  1981: V
Episode  1982: V
Episode  1983: VD
Episode  1984: V
Episode  1985: VD
Episode  1986: V
Episode  1987: VVV
Episode  1988: V
Episode  1989: VV
Episode  1990: V
Episode  1991: V
Episode  1992: V
Episode  1993: D
Episode  1994: VV
Episode  1995: D
Episode  1996:

Episode  2873: V
Episode  2874: V
Episode  2875: V
Episode  2876: V
Episode  2877: V
Episode  2878: V
Episode  2879: V
Episode  2880: V
Episode  2881: V
Episode  2882: V
Episode  2883: V
Episode  2884: V
Episode  2885: V
Episode  2886: V
Episode  2887: VD
Episode  2888: D
Episode  2889: D
Episode  2890: V
Episode  2891: V
Episode  2892: V
Episode  2893: V
Episode  2894: V
Episode  2895: V
Episode  2896: V
Episode  2897: V
Episode  2898: V
Episode  2899: V
Episode  2900: V
Episode  2901: V
Episode  2902: VV
Episode  2903: V
Episode  2904: V
Episode  2905: V
Episode  2906: V
Episode  2907: V
Episode  2908: VVD
Episode  2909: V
Episode  2910: V
Episode  2911: D
Episode  2912: V
Episode  2913: V
Episode  2914: V
Episode  2915: V
Episode  2916: VVVVD
Episode  2917: V
Episode  2918: V
Episode  2919: D
Episode  2920: VVD
Episode  2921: V
Episode  2922: V
Episode  2923: V
Episode  2924: V
Episode  2925: V
Episode  2926: V
Episode  2927: V
Episode  2928: V
Episode  2929: V
Episode  2930: V
Epis

...

Episode  4557: VD
Episode  4558: V
Episode  4559: V
Episode  4560: V
Episode  4561: V
Episode  4562: V
Episode  4563: VV
Episode  4564: V
Episode  4565: V
Episode  4566: V
Episode  4567: V
Episode  4568: V
Episode  4569: VD
Episode  4570: D
Episode  4571: V
Episode  4572: V
Episode  4573: V
Episode  4574: VV
Episode  4575: V
Episode  4576: V
Episode  4577: V
Episode  4578: VD
Episode  4579: V
Episode  4580: V
Episode  4581: V
Episode  4582: V
Episode  4583: V
Episode  4584: V
Episode  4585: V
Episode  4586: V
Episode  4587: V
Episode  4588: VVVD
Episode  4589: V
Episode  4590: V
Episode  4591: V
Episode  4592: VD
Episode  4593: V
Episode  4594: V
Episode  4595: V
Episode  4596: V
Episode  4597: VD
Episode  4598: V
Episode  4599: V
Episode  4600: V
Episode  4601: V
Episode  4602: V
Episode  4603: V
Episode  4604: V
Episode  4605: V
Episode  4606: V
Episode  4607: V
Episode  4608: V
Episode  4609: V
Episode  4610: V
Episode  4611: V
Episode  4612: V
Episode  4613: V
Episode  4614: V
Epis

Episode  5269: VV
Episode  5270: V
Episode  5271: VD
Episode  5272: V
Episode  5273: V
Episode  5274: VD
Episode  5275: V
Episode  5276: V
Episode  5277: V
Episode  5278: VD
Episode  5279: V
Episode  5280: V
Episode  5281: V
Episode  5282: V
Episode  5283: V
Episode  5284: V
Episode  5285: VVD
Episode  5286: VD
Episode  5287: V
Episode  5288: V
Episode  5289: V
Episode  5290: V
Episode  5291: V
Episode  5292: V
Episode  5293: V
Episode  5294: D
Episode  5295: V
Episode  5296: VVV
Episode  5297: VD
Episode  5298: VD
Episode  5299: V
Episode  5300: VD
Episode  5301: V
Episode  5302: V
Episode  5303: VD
Episode  5304: V
Episode  5305: V
Episode  5306: V
Episode  5307: V
Episode  5308: V
Episode  5309: V
Episode  5310: V
Episode  5311: V
Episode  5312: V
Episode  5313: V
Episode  5314: V
Episode  5315: V
Episode  5316: V
Episode  5317: VVD
Episode  5318: D
Episode  5319: V
Episode  5320: V
Episode  5321: V
Episode  5322: V
Episode  5323: V
Episode  5324: V
Episode  5325: V
Episode  5326: V

Episode  5770: VD
Episode  5771: V
Episode  5772: VD
Episode  5773: V
Episode  5774: V
Episode  5775: V
Episode  5776: VD
Episode  5777: V
Episode  5778: V
Episode  5779: V
Episode  5780: V
Episode  5781: V
Episode  5782: V
Episode  5783: V
Episode  5784: V
Episode  5785: V
Episode  5786: V
Episode  5787: D
Episode  5788: V
Episode  5789: V
Episode  5790: VD
Episode  5791: D
Episode  5792: VD
Episode  5793: V
Episode  5794: V
Episode  5795: V
Episode  5796: V
Episode  5797: V
Episode  5798: VD
Episode  5799: V
Episode  5800: V
Episode  5801: V
Episode  5802: V
Episode  5803: V
Episode  5804: V
Episode  5805: V
Episode  5806: V
Episode  5807: VVD
Episode  5808: V
Episode  5809: V
Episode  5810: V
Episode  5811: V
Episode  5812: V
Episode  5813: VD
Episode  5814: D
Episode  5815: V
Episode  5816: V
Episode  5817: V
Episode  5818: V
Episode  5819: V
Episode  5820: V
Episode  5821: V
Episode  5822: V
Episode  5823: V
Episode  5824: D
Episode  5825: V
Episode  5826: V
Episode  5827: V
Episo

Episode  6265: V
Episode  6266: V
Episode  6267: V
Episode  6268: V
Episode  6269: D
Episode  6270: V
Episode  6271: VD
Episode  6272: VV
Episode  6273: D
Episode  6274: V
Episode  6275: V
Episode  6276: V
Episode  6277: V
Episode  6278: V
Episode  6279: V
Episode  6280: V
Episode  6281: V
Episode  6282: V
Episode  6283: V
Episode  6284: V
Episode  6285: V
Episode  6286: V
Episode  6287: V
Episode  6288: V
Episode  6289: V
Episode  6290: V
Episode  6291: V
Episode  6292: V
Episode  6293: D
Episode  6294: V
Episode  6295: V
Episode  6296: V
Episode  6297: VD
Episode  6298: D
Episode  6299: V
Episode  6300: V
Episode  6301: V
Episode  6302: V
Episode  6303: V
Episode  6304: V
Episode  6305: V
Episode  6306: V
Episode  6307: V
Episode  6308: V
Episode  6309: VD
Episode  6310: V
Episode  6311: V
Episode  6312: V
Episode  6313: D
Episode  6314: VVV
Episode  6315: D
Episode  6316: V
Episode  6317: V
Episode  6318: V
Episode  6319: V
Episode  6320: VD
Episode  6321: V
Episode  6322: V
Episode

Episode  6768: VVVVVD
Episode  6769: V
Episode  6770: V
Episode  6771: V
Episode  6772: V
Episode  6773: VD
Episode  6774: V
Episode  6775: VD
Episode  6776: VD
Episode  6777: V
Episode  6778: V
Episode  6779: V
Episode  6780: V
Episode  6781: V
Episode  6782: VVD
Episode  6783: V
Episode  6784: D
Episode  6785: V
Episode  6786: V
Episode  6787: VD
Episode  6788: VVD
Episode  6789: V
Episode  6790: V
Episode  6791: VD
Episode  6792: V
Episode  6793: V
Episode  6794: VD
Episode  6795: VV
Episode  6796: V
Episode  6797: V
Episode  6798: V
Episode  6799: V
Episode  6800: V
Episode  6801: V
Episode  6802: V
Episode  6803: V
Episode  6804: VVV
Episode  6805: D
Episode  6806: V
Episode  6807: V
Episode  6808: V
Episode  6809: V
Episode  6810: VVV
Episode  6811: V
Episode  6812: V
Episode  6813: V
Episode  6814: D
Episode  6815: V
Episode  6816: V
Episode  6817: V
Episode  6818: V
Episode  6819: V
Episode  6820: V
Episode  6821: VD
Episode  6822: V
Episode  6823: V
Episode  6824: V
Episode  6